# Theoretical Headway

In [ ]:
import pandas as pd

In [ ]:
calendar_dates = pd.read_csv('data/timetables/gtfs3Sept/calendar_dates.txt', sep = ",")
calendar = pd.read_csv('data/timetables/gtfs3Sept/calendar.txt', sep = ",")
trips = pd.read_csv('data/timetables/gtfs3Sept/trips.txt', sep = ",")
stops = pd.read_csv('data/timetables/gtfs3Sept/stops.txt', sep = ",")

stop_times = pd.read_csv('data/timetables/gtfs3Sept/stop_times.txt', sep = ",")

routes = pd.read_csv('data/timetables/gtfs3Sept/routes.txt', sep = ",")

In [ ]:
stop = stops.head(1).to_dict(orient="records")[0]
stop

In [ ]:
stop_times_stop = stop_times.merge(
    stops,
    "left",
    "stop_id"
)

In [ ]:
stop_times_stop = stop_times_stop[stop_times_stop.stop_id == stop.get("stop_id")]
stop_times_stop

In [ ]:
stop_times_with_trips = stop_times_stop.merge(
    trips,
    "left",
    "trip_id"
)
route_id = stop_times_with_trips.head(1).to_dict("records")[0].get("route_id")

In [ ]:
stop_times_with_trips = stop_times_with_trips[stop_times_with_trips.route_id == route_id]
stop_times_with_trips

In [ ]:
# Fix direction
direction_id = stop_times_with_trips.head(1).to_dict("records")[0].get("direction_id")

# Fix Service
service_id = stop_times_with_trips.head(1).to_dict("records")[0].get("service_id")

print(direction_id, service_id)

In [ ]:
stop_times_with_trips_stops = stop_times_with_trips[(stop_times_with_trips.direction_id == direction_id) & (stop_times_with_trips.service_id == service_id)]

In [ ]:
time = stop_times_with_trips_stops["arrival_time"].str.split(':', expand = True)
t = time[0].astype(int) * 3600 + time[1].astype(int) * 60 + time[2].astype(int)
stop_times_with_trips_stops["time_seconds"] = t

In [ ]:
stop_times_with_trips_stops = stop_times_with_trips_stops.sort_values(by=["time_seconds"], ascending=True)

In [ ]:
stop_times_with_trips_stops

In [ ]:
stop_times_with_trips_stops["headway"] = (stop_times_with_trips_stops.time_seconds.sort_values() - stop_times_with_trips_stops.time_seconds.sort_values().shift()) / 60

In [ ]:
stop_times_with_trips_stops.sort_values(by="time_seconds", ascending= True)[["trip_id", "arrival_time","headway", "stop_name", "route_id", "service_id", "trip_headsign"]]

In [ ]:
import plotly.express as px
fig = px.scatter(stop_times_with_trips_stops.sort_values("time_seconds"), x="arrival_time", y="headway", color='headway',  marginal_y="histogram", text="headway")
fig.show()

# Clustering

### Hierarchical agglomerative clustering 
<br> https://towardsdatascience.com/machine-learning-algorithms-part-12-hierarchical-agglomerative-clustering-example-in-python-1e18e0075019
<br> https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
<br> https://analyticsindiamag.com/all-you-need-to-know-about-time-series-clustering/

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

In [ ]:
dataset = stop_times_with_trips_stops.sort_values(by="time_seconds", ascending= True)[["trip_id", "time_seconds","headway", "stop_name", "route_id", "service_id", "trip_headsign"]]

In [ ]:
dataset.fillna(0)

In [ ]:
dataset["time_seconds"] = dataset["time_seconds"].fillna(0.0).astype(int)

In [ ]:
dataset

In [ ]:
data = dataset.iloc[:, [1, 2]].values

In [ ]:
data 

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(data, method='ward'))

In [ ]:
model = AgglomerativeClustering(n_clusters= None,affinity='euclidean', linkage='ward', distance_threshold=10)
model.fit(data)
labels = model.labels_

In [ ]:
plt.scatter(data[labels==0, 0], data[labels==0, 1], s=50, marker='o', color='red')
plt.scatter(data[labels==1, 0], data[labels==1, 1], s=50, marker='o', color='blue')
plt.scatter(data[labels==2, 0], data[labels==2, 1], s=50, marker='o', color='green')
plt.scatter(data[labels==3, 0], data[labels==3, 1], s=50, marker='o', color='purple')
plt.scatter(data[labels==4, 0], data[labels==4, 1], s=50, marker='o', color='orange')
plt.show()

éclaté enft 